In [1]:
import pandas as pd

In [2]:
df_line_all = pd.read_csv("./df_line_all_20230317.csv",index_col = 0)
#Select the required information from the information summary table
df_line = df_line_all.loc[:,df_line_all.columns.tolist()[0:42]+df_line_all.columns.tolist()[64:78]]

In [6]:
def cu_poltim(df,listtim = [19,0.7,2.2,4,7],Cruise=0):
    
    df_base = df
    
    #Build input (common name) - query (EEDB) mapping
    list_pol = ["HC","CO","NOx","SO2","nvPM","CO2","nvPM number"]
    list_stage = ["Taxi-off","Take-off","Climb-out","Cruise","Approach","Taxi-in"]
    list_pol_c = ["HC","CO","NOx","SOx","nvPM EImass","CO2","nvPM EInum"]
    list_stage_c = ["Idle","T/O","C/O","Cruise","App","Idle"]
    df_c = pd.Series(list_pol_c+list_stage_c,index = list_pol+list_stage)
    #Classification of pollutants
    list1 = ["HC","CO","NOx"]
    list2 = ["SO2"]
    list3 = ["nvPM"]
    list4 = ["nvPM number"]
    list5 = ["CO2"]
    #Time (other than cruise)
    df_base["Taxi-off"+" TIM"] = listtim[0]
    df_base["Take-off"+" TIM"] = listtim[1]
    df_base["Climb-out"+" TIM"] = listtim[2]
    df_base["Approach"+" TIM"] = listtim[3]
    df_base["Taxi-in"+" TIM"] = listtim[4]
    #Cruise time reduction function
    def yy(x):
        if x>=Cruise:
            x=x-Cruise
        else:
            x=x/Cruise
        return x
    #Cruise time
    df_base["Cruise TIM"] = df_base["Cruise TIM"].astype(int).apply(lambda x:yy(x))
    
    #FF&EF
    #pollutions
    for pol in list_pol:
        #phases
        for stage in list_stage:
            if pol in list1:
                ser_ei = df_base[df_c[pol]+" EI "+df_c[stage]+" (g/kg)"]
            if pol in list2:
                ser_ei = df_base["Fuel Sulphur Content (ppm by mass)"]/1000*2*0.967#ppm(mg/kg)-g/kg
            if pol in list3:
                ser_ei = df_base[df_c[pol]+" "+df_c[stage]+" (mg/kg)"]/1000#g/kg
            if pol in list4:
                ser_ei = df_base[df_c[pol]+" "+df_c[stage]+" (#/kg)"]#数量
            if pol in list5:
                ser_ei = 3155
            #volume
            nofa = df_base["航班量"]
            #phase time
            tim = df_base[stage+" TIM"]
            #number of engine
            enn = df_base["Engine number"]
            #Fuel Flow
            fuel = df_base["Fuel Flow "+df_c[stage]+" (kg/sec)"]
            #Calculation (by line, phase * pollutant), flight volume * flight time (min) * fuel flow rate (kg/sec) * engine number * emission index (g/kg)
            #nvPM number in "#", other units in "g"
            if pol == "nvPM number":
                df_base[pol+" "+stage] = nofa*tim*60*fuel*enn*ser_ei
            else:
                df_base[pol+" "+stage+" (g)"] = nofa*tim*60*fuel*enn*ser_ei
#            print(pol,stage,"finished")
# Consolidation phases by pollution
    for pol in list_pol:
        if pol == "nvPM number":
            df_base[pol] = df_base[pol+" Taxi-off"]+df_base[pol+" Take-off"]+df_base[pol+" Climb-out"]+df_base[pol+" Cruise"]+df_base[pol+" Approach"]+df_base[pol+" Taxi-in"]
        else:
            df_base[pol] = df_base[pol+" Taxi-off"+" (g)"]+df_base[pol+" Take-off"+" (g)"]+df_base[pol+" Climb-out"+" (g)"]+df_base[pol+" Cruise"+" (g)"]+df_base[pol+" Approach"+" (g)"]+df_base[pol+" Taxi-in"+" (g)"]
    list_pol0 = ["HC","CO","NOx","SO2","nvPM","CO2"]
    #Output
    return [[pol,df_base[pol].sum()] for pol in list_pol0]
cu_poltim(df_line,[19,0.7,2.2,4,7])

[['HC', 8080126853.437977],
 ['CO', 177547565879.89215],
 ['NOx', 302030276473.4495],
 ['SO2', 23098104899.91732],
 ['nvPM', 990259244.125932],
 ['CO2', 87160641841837.45]]